In [4]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Kalah Rules

### Game structure:
<p>The playing field consists of 6 houses per side and additionally one large house per player at the edge, the so-called storage. Six seeds are placed in each house except the storage.</p>

![title](images/Board.png)

### Gameplay:
<p>Alternately, a player selects a house on his side of the board. The player then places the seeds in the following houses in a counter-clockwise direction. A seed is also placed in the own storage, the opponent's storage is left out. If the last seed is placed in the own storage, it is the player's turn again. If the last seed is placed in an empty house on the player's own side of the board, this seed and all the seeds in the opposite house are placed in the player's storage. In that case, it is the opponent's turn.</p>

### End of the game:
<p>The game ends when all the houses of one player have been emptied. The opposing player then places all remaining seed in his storage.</p>

### Object of the game:
<p>The winner is the player who has more seeds in his storage at the end of the game.</p>

source: http://www.kalaha.de/kalaha.htm Abruf am 9.11.2021

# Classes

The Kalah game is implemented with the use of several classes, containing base class **Kalah_Game**, a **Board** class for displaying the UI and a **Player** class as well as several classes inheriting from this **Player** class:

- **Human**
- **Random_KI**

The following illustration shows a class diagram of all classes involved:

<img src="images/Kalah_Class_Diagram.png" alt="Kalah_Class_Diagram" width="600"/>

This image was created with [Creatly](https://creately.com/).

## Global Variables

Additionally to the classes, there are two global variables with the purpose to help converting the state of the game board, which is represented by a nested list, to letters. The letter representation is mainly for the UI, but also distributes to print and logging messages being easier to humanly comprehend.

In [5]:
order = [['A','B','C','D','E','F','O'],
         ['a','b','c','d','e','f','o']]
letter_numbers = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5 }

## Board Class

The class **Board** has the purpose to create the game UI by drawing a Kalah board if requested.

#### Methods:

- *_state_to_dict(state)*
- *draw(state)*
- *drawBackground*

### _state_to_dict

The private method **_state_to_dict** takes a board state in form of a nested list and transforms it into a dictionary with the letter representation of the troughes as keys and the trough values from the list as values. This representation is only needed for the draw function.

### draw

The method **draw** takes a board state and creates an illustration of the Kalah board by using the library <tt>ipycanvas</tt>. 

### _drawBackground

The private method **drawBackground** takes no parameters and draws the base for the game board. Since we are using a Multi-layered Canvas we use this function to draw the basic board as a background on the lowest layer of the MultiCanvas. This enables us to avoid redrawing the entirety of the board after every change and instead only redraw elements that are subject to change.

In [6]:
from ipycanvas import Canvas
from ipycanvas import MultiCanvas

In [7]:
class Board:
    multi_canvas = MultiCanvas(width=800, height=350)
    
    def __init__(self):
        
        self._drawBackground()
    
    def _state_to_dict(self, state):
        d = {}
        for i in [0,1]:
            for j in range(7):
                d[order[i][j]] = state[i][j]
        return d
    
    def _drawBackground(self):
        self.multi_canvas[0]
        self.multi_canvas[0].fill_style='brown'
        self.multi_canvas[0].fill_rect(0,0,self.multi_canvas.width)

        self.multi_canvas[0].stroke_circles([((i)*100)+145 for i in range(6)],[75 for i in range(6)], 40)
        self.multi_canvas[0].stroke_circles([(i*100)+145 for i in range(6)],[255 for i in range(6)], 40)

        self.multi_canvas[0].stroke_rects([10,self.multi_canvas.width-100],[85,85],[80,80],[180,180])

        self.multi_canvas[0].fill_style='white'
        self.multi_canvas[0].font = '32px serif'
        self.multi_canvas[0].fill_text("O",40,70)
        self.multi_canvas[0].fill_text("F",135,30)
        self.multi_canvas[0].fill_text("E",235,30)
        self.multi_canvas[0].fill_text("D",335,30)
        self.multi_canvas[0].fill_text("C",435,30)
        self.multi_canvas[0].fill_text("B",535,30)
        self.multi_canvas[0].fill_text("A",635,30)
        # ---------
        self.multi_canvas[0].fill_style = 'black'
        self.multi_canvas[0].fill_text("a",135,320)
        self.multi_canvas[0].fill_text("b",235,320)
        self.multi_canvas[0].fill_text("c",335,320)
        self.multi_canvas[0].fill_text("d",435,320)
        self.multi_canvas[0].fill_text("e",535,320)
        self.multi_canvas[0].fill_text("f",635,320)
        self.multi_canvas[0].fill_text("o",735,290)
    
    def draw(self, state):
        house = self._state_to_dict(state)
        self.multi_canvas[1].clear()
        self.multi_canvas[1].font = '32px serif'
        self.multi_canvas[1].fill_style = 'black'
        
        self.multi_canvas[1].fill_text(str(house['F']),135,140)
        self.multi_canvas[1].fill_text(str(house['E']),235,140)

        self.multi_canvas[1].fill_text(str(house['D']),335,140)
        self.multi_canvas[1].fill_text(str(house['C']),435,140)
        self.multi_canvas[1].fill_text(str(house['B']),535,140)
        self.multi_canvas[1].fill_text(str(house['A']),635,140)

        self.multi_canvas[1].fill_text(str(house['a']),135,210)
        self.multi_canvas[1].fill_text(str(house['b']),235,210)
        self.multi_canvas[1].fill_text(str(house['c']),335,210)
        self.multi_canvas[1].fill_text(str(house['d']),435,210)
        self.multi_canvas[1].fill_text(str(house['e']),535,210)
        self.multi_canvas[1].fill_text(str(house['f']),635,210)

        self.multi_canvas[1].fill_text(str(house['o']),730,175)
        self.multi_canvas[1].fill_text(str(house['O']),40,175)

        return self.multi_canvas

In [8]:
from ipycanvas import Canvas

In [10]:
Board().draw([(1,6,8,0,20,6,17), (0,0,4,18,2,1,3)])

Canvas(height=350, width=800)

# Player Class

#### Attributes:
- *number*
- *name*

#### Methods:

- *\_\_init__(number, name)*
- *\_\_str__()*
- *_available_house(own_house)*
- *choose_house(current_state)*

The class **Player** is the superclass which represents a Kalah game player. In Kalah there are exactly two players which compete against each other. 

The attribute *number* of a player has either the value 0 or 1 depending on the order in which the players take their turns.
If the number of the player is 0, they start the game by having the first turn.
Additionally, *number* represents the index of the state list which contains the player's list of house values.

The attribute *name* is a string which represents the player's name in the UI or print and logging messages.

### Method: __init__
The method *\_\_init__(number, name)* initializes the **Player** object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised.

### Method: __str__
The method *\_\_str__()* defines the *name* of the **Player** object as their string representative for print messages.

In [11]:
class Player:
    
    def __init__(self, number, name):
        if number in [0,1]:
            self.number = number
        else:
            raise ValueError("Number of player must be 0 or 1!")
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
            
    def __str__(self):
        return self.name

### Method: _available_house

The private method *_available_house(own_house)* receives a list which represents the player's house and their store. The method returns a list with the indizes of all house which contain at least one seed (list indizes with a value higher than null).

In [12]:
def _available_house(self, own_house):
    available = []
    for i in range(len(own_house)):
        if own_house[i] != 0:
            available.append(i)
    return available

Player._available_house = _available_house
del _available_house

### Method: choose_house

The method *choose_house(current_state)* receives the current state of the Kalah board and returns the index of the chosen house from the player's house list. This method differentiates the **Player** subclasses and is therefore differently implemented in each of them. There is no basic implementation of this method in the **Player** class. This is why in the class **Kalah_Game** only subclasses of **Player** are accepted as players. Therefore, this class is intended as an abstract class.

In [13]:
def choose_house(self, current_state):
    pass

Player.choose_house = choose_house
del choose_house

## Tests of Player Class

### 1. Player number must be 0 or 1

In [14]:
try:
    Player(2, "Player1")
except ValueError as e:
    print(e)

Number of player must be 0 or 1!


### 2. Player name must be a string

In [15]:
try:
    Player(0, 3)
except ValueError as e:
    print(e)

Name must be a string!


### 3. Successful Player creation

In [16]:
Player(0, "Player1")

## Human Player Class

The class **Human** inherits from **Player**.
It is used for humans playing against each other or against one of the KIs. 

### Method: choose_house
The method *choose_house(current_state)* is implemented as follows:

At first the own house are extracted from the game state and the available house indizes are calculated using the method *_available_house*. Afterwards the human player is asked to choose one of the available house via a input field. With limiting the player to the available house, it is easier to detect a correct input. If the player enters an unvalid value, they are asked to input a value again until a valid value is given. The index of the corresponding house is returned.

In [17]:
class Human(Player):
    
    def choose_house(self, current_state):
        own_t,store = current_state[self.number][:6],current_state[self.number][6]
        available = self._available_house(own_t)
        
        i_string = "Choose one of the available house:\n"
        for i in available:
            i_string += f"{order[self.number][i]}, "
        
        choice_str = ""
        while choice_str not in [k for k in letter_numbers if letter_numbers[k] in available]:
            choice_str = input(i_string[:-2]+"\n").lower()
        
        choice = letter_numbers[choice_str]
        return choice

### Test of Human Player Class

In [18]:
human = Human(1,"Hans")

In [19]:
#human.choose_house([[6,6,6,6,6,6,0], [6,0,6,6,4,6,0]])

## Random_KI Player Class

The class **Random_KI** inherits from **Player**. It is a simple KI player implementation which chooses house at random.

The class **Random_KI** has a modified *\_\_init__* function which has an additional argument *seed* for setting a seed for the <tt>random</tt> library functions. Whenever the same seed is set, the <tt>random</tt> function creates the same random numbers or chooses the same random objects from a list. Adding this argument eases the logging and debbuging of **Random_KI** players.

In [20]:
import random as rn

class Random_KI(Player):
    
    def __init__(self, number, name, seed):
        rn.seed(seed)
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* is implemented as follows:

At first the own house are extracted from the game state and the available house indizes are calculated using the method *_available_house*. From the list of available house indizes, one is chosen at random using the function <tt>choice</tt> from the library <tt>random</tt>. Afterwards, this value is returned.

In [21]:
def choose_house(self, current_state):
    own_t,safe = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_house(own_t)

    choice = rn.choice(available)

    return choice

Random_KI.choose_house = choose_house
del choose_house

### Test of Random_KI Player Class

In [22]:
ki = Random_KI(0,"Rando", 1)

In [23]:
ki.choose_house([[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]])

1

# Kalah_Game Class

#### Attributes:
- *state*
- *board*
- *players*
- *current_player*

#### Methods:

- *\_\_init__(players)*
- *_other_player(player_num)*
- *_show_state(state)*
- *show_state()*
- *draw_board()*
- *_move(player_num, choice)*
- *_finished()*
- *_calculate_winner()*
- *start()*
- *utility(player_num)*

The class **Kalah_Game** is the core of the Kalah game implementation. It contains all information on the game, including the current game state.

The attribute *state* represents the state of the Kalah board which is defined by the number of seeds laying in each of the player's house and their stores. It is implemented by a nested list which contains a list for the house on each of the two players' sides. The last index of each of the lists is the store of that player. At the start of the game, where the stores are empty and there are six seeds in every house, the implemented representation of the state for example is: [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]].

The attribute *board* is an instance of the earlier described **Board** class and has the purpose to draw the game UI.

The attribute *players* is a list of the two players that play the game. They must be instances of a subclass of the **Player** class. The order in which they take turns is determined by their **Player** *number*: The player with the number 0 goes first.

The attribute *current_player* has either the value 0 or 1. It takes track of which player's turn it currently is. If *current_player* has the value 0 for example, it is the turn of the player which stands at index 0 of the *players* list and therefore also has the **Player**'s class attribute *number* of value 0.

### Method: __init__
The method *\_\_init__(players)* initializes the game by setting the *state* to the initial state (seen above), initializing a board object and setting the *players* list using the received "players" argument. Before setting the *players* list, the received list is checked for the number of items it contains (must be exactly 2) and if the items in the list are both instances of a **Player** subclass (but not of the class **Player** itself). In error case, matching error messages are raised.

In [24]:
class Kalah_Game():
    
    def __init__(self, players):
        self.state = [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]]
        self.board = Board()
        
        if len(players) != 2:
            raise ValueError("There must be exactly two players!")
        if not ((isinstance(players[0], Player) and type(players[0]) != Player) 
            and (isinstance(players[1], Player) and type(players[1]) != Player)):
            raise ValueError("Both players must be of instances of a subclass of the class Player!")
        if {players[0].number, players[1].number} != {0,1}:
            raise ValueError("One of the players must be self.number 0 and the other one self.number 1!")
        
        self.players = players
        self.current_player = 0

### Method: _other_player
The auxiliary method *_other_player(player_num)* returns the opponent's player number to a given player number. This means that it returns 1 for the input 0, and 0 for the input 1.

In [25]:
def _other_player(self, player_num):
    return (player_num + 1) % 2

Kalah_Game._other_player = _other_player
del _other_player

### Methods: _show_state and show_state
The private method *_show_state(state)* creates a formatted string which represents the received state and prints it to the console. It can be used as an alternative to the <tt>ipycanvas</tt> game UI.

The method *show_state()* calls the private mthod *_show_state(state)* with the current game state (attribute *state*).

In [26]:
def _show_state(self, state):
    s = f''

    s += f'{self.players[0].name}:\t\t\t'
    for j in range(6,-1,-1):
        s += f'{order[0][j]}: {state[0][j]}  '
    s += f'\n'

    s += f'{self.players[1].name}:\t\t\t'
    for j in range(7):
        s += f'{order[1][j]}: {state[1][j]}  '
    s += f'\n'

    print(s)

Kalah_Game._show_state = _show_state
del _show_state

In [27]:
def show_state(self):
    self._show_state(self.state)
    
Kalah_Game.show_state = show_state
del show_state

### Method: draw_board
The method *draw_board()* calls the *draw(state)* method of the *board* instance with the current game state (attribute *state*).

In [28]:
def draw_board(self):
    return self.board.draw(self.state)

Kalah_Game.draw_board = draw_board
del draw_board

### Method: _move
The private method *_move(player_num, choice)* receives the number of the current player and the house index they have chosen as a result of the **Player** method *choose_house*. It calculates the actions of the player's turn and returns the resulting new game state.

##### The calculations are implemented based on the following game rules:

1. The current player choses one of his house. The seeds from the chosen house are placed counterclockwise in the house of both players and in the store of the current player. The store of the opponent is left out. Then it is the turn of the opponent.


2. If the last seed is placed in the store of the current player, they get another move.


3. If the last seed is placed in an empty house of the current player the seed from this house and all seeds from the opposite house are place in the store of the current player. Then it's the opponent's turn.

In [29]:
def _move(self, choice):
    new_state = self.state.copy()

    seeds = self.state[self.current_player][choice]
    new_state[self.current_player][choice] = 0

    c_house_player_num = self.current_player
    c_house_num = choice

    # Go through houses counterclockwise
    while (seeds > 0):
        c_house_num += 1

        # Skip opponent's store
        if(c_house_num == 6 and c_house_player_num != self.current_player):
            continue

        # Switch to houses of the other player
        if(c_house_num > 6):
            c_house_num = 0
            c_house_player_num = self._other_player(c_house_player_num)

        # Add seed to the currently visited house
        new_state[c_house_player_num][c_house_num] += 1
        seeds -= 1

    # Check for special rules after last seed was placed in own store or own empty house:
    if(c_house_player_num == self.current_player):

        # Rule: Another turn if last seed is placed in own store
        if(c_house_num == 6):
            # Give current player another turn
            print(f'Player {self.players[self.current_player].name} gets an extra turn!')
            # Twist the order beforehand so the standard game loop grants the extra turn automatically
            self.current_player = self._other_player(self.current_player)

        # Rule: Last seed is placed in empty house of current player
        elif(new_state[self.current_player][c_house_num] == 1):                
            # Collect all seeds to be rewarded and empty both houses
            receivedSeeds = new_state[self.current_player][c_house_num]
            receivedSeeds += new_state[self._other_player(self.current_player)][5 - c_house_num]
            new_state[self._other_player(self.current_player)][5 - c_house_num] = 0
            new_state[self.current_player][c_house_num] = 0
            # Award all the seeds to the current player's kalah
            new_state[self.current_player][6] += receivedSeeds
            print(f'Player {self.players[self.current_player].name} gets a steal for {receivedSeeds} Seeds!')

    return new_state

Kalah_Game._move = _move
del _move

### Method: _finished
The private method *_finished()* checks if one of the players has no seeds in their house left and is therefore unable to take another turn. If this is the case, the method returns True, otherwise it returns False. 

In [30]:
def _finished(self):
    sum0 = sum(self.state[0][:-1])
    sum1 = sum(self.state[1][:-1])

    if(not (sum0 == 0 or sum1 == 0)):
        return False
    self.show_state()
    print("Finished Game!")
    return True

Kalah_Game._finished = _finished
del _finished

### Method: _calculate_winner
The private method *_calculate_winner()* counts the seeds on each player's side of the board (house and store). The player with the higher number of seeds wins the game. If both players have the same number of seeds, the game ends with a draw. The method prints the winner to the console.

In [31]:
def _calculate_winner(self):
    sum0 = sum(self.state[0])
    sum1 = sum(self.state[1])
    print(f"{self.players[0]}: {sum0} Points. {self.players[1]}: {sum1} Points.")
    if(sum0 > sum1):
        print(f"{self.players[0]} wins!")
    elif(sum1 > sum0):
        print(f"{self.players[1]} wins!")
    else:
        print(f"Draw!")
        
Kalah_Game._calculate_winner = _calculate_winner
del _calculate_winner

### Method: start
The method *start()* starts the Kalah game. Until the game is finished (the method *_finished()* returns True), both players take turns, starting with the player with the number 0. At the start of each turn, the current game state is shown. Next, the current player chooses one of the house with the **Player** method *choose_house(current_state)*. Afterwards, this choice is handed to the private method *_move(player_num, choice)* which calculates the new game state. The attribute *state* is updated with this new game state. Then it is the turn of the other player. If the game is finished, the method *_calculate_winner()* calculates which of the players wins the game and prints the result to the console.

In [32]:
def start(self):
    while(not self._finished()):
        print("Current state:")
        self.show_state()
        print(f"Next is {self.players[self.current_player].name}'s turn.")

        choice = self.players[self.current_player].choose_house(self.state)
        print(f"{self.players[self.current_player].name} chose {order[self.current_player][choice]}")

        self.state = self._move(choice)

        self.current_player = self._other_player(self.current_player)

    self._calculate_winner()
    
Kalah_Game.start = start
del start

### Method: utility
The  method *utility(player_num)* receives the number of the current player and uses the current state to calculate the utility of the state for the player. The method compares the seeds from both stores and returns an Integer dependent on the result. If the current player has more seeds than the opponent the method returns 1. If they own less seeds than the opponent the method returns -1 and if it is a draw the number 0 is returned.     

In [33]:
def utility(self, player_num):
    playerStore = self.state[player_num][6]
    opponentStore = self.state[self._other_player(player_num)][6]
    if(playerStore > opponentStore):
        return 1
    elif(playerStore == opponentStore):
        return 0
    else:
        return -1
    
Kalah_Game.utility = utility
del utility

## Tests of Kalah_Game Class

### 1. There must be exactly two players

In [ ]:
try:
    game = Kalah_Game([Player(0,"Test")])
except ValueError as e:
    print(e)

### 2. Players must be instances of a Player sublass

In [ ]:
try:
    game = Kalah_Game([Player(0,"Test"), Player(1,"Test")])
except ValueError as e:
    print(e)

### 3. Player numbers must be 0 and 1

In [ ]:
try:
    game = Kalah_Game([Human(1,"Human"), Random_KI(1,"KI",2)])
except ValueError as e:
    print(e)

### 4. Successful Game creation with two KIs

In [ ]:
game = Kalah_Game([Random_KI(0,"KI1",1), Human(1,"Karl")])

In [ ]:
game.show_state()

In [ ]:
game.start()

In [ ]:
gameHuman = Kalah_Game([Random_KI(0,"KI2",1), Human(1,"Human")])

In [ ]:
gameHuman.show_state()

In [ ]:
gameHuman.start()